In [ ]:
%matplotlib ipympl

import dash
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### dash.HDF5DAS inherits from h5py.File
#### You can open and close a file manually

In [ ]:
f5 = dash.HDF5DAS("DAS/continuous_denoised.hdf5", mode="r")
f5.close()

#### Or automatically close the file when done by using a context manager

In [ ]:
with dash.HDF5DAS("DAS/continuous_denoised.hdf5", mode="r") as f5:
    pass

#### Use the dash.HDF5DAS.gather() method to read waveforms

In [ ]:
with dash.HDF5DAS("DAS/continuous_denoised.hdf5", mode="r") as f5:
    gather = f5.gather(
        "2017-03-26T18:00:29.4Z", 
        "2017-03-26T18:00:32.5Z"
    )

#### Waveforms are returned as a dash.Gather object with helpful properties and methods
##### Like the waveform data as a 2D NumPy array...

In [ ]:
gather.data

##### Or as a pandas.DataFrame

In [ ]:
gather.dataf

##### Or as an obspy.Stream

In [ ]:
gather.datas

##### The start and end times

In [ ]:
print(gather.starttime)
print(gather.endtime)

##### The time of each sample

In [ ]:
gather.times

##### Sampling rate, interval, and Nyquist frequency

In [ ]:
print(gather.sampling_rate)
print(gather.sampling_interval)
print(gather.nyquist)

##### The number of traces (x-axis) and samples (t-axis)

In [ ]:
print(gather.ntraces)
print(gather.nsamples)

#### The data can be easily plotted in the time domain

In [ ]:
gather.plot()

#### Or the fk domain

In [ ]:
gather.plot(domain="fk")

#### And you can set the colormap to your preference

In [ ]:
gather.plot(cmap=plt.get_cmap("seismic"))

#### You can read subsets of traces from disk

In [ ]:
with dash.HDF5DAS("DAS/continuous_denoised.hdf5", mode="r") as f5:
    gather = f5.gather(
        "2017-03-26T18:00:29.4Z", 
        "2017-03-26T18:00:32.5Z",
        traces=slice(None, None, 100)
    )
    
gather.plot()

#### You can write to disk in HDF5 format

In [ ]:
gather.write("test.hdf5")

#### Or in raw binary format

In [ ]:
gather.write_binary("test.bin")

#### And you can process the data

In [ ]:
with dash.HDF5DAS("DAS/continuous_raw.hdf5", mode="r") as f5:
    gather = f5.gather(
        "2017-03-26T18:00:29.4Z", 
        "2017-03-26T18:00:32.5Z"
    )
    
gather.plot()

In [ ]:
gather.demean()
gather.decimate(4)
gather.bandpass(locut=16, hicut=256)
gather.normalize()
FK_MASK              = dash.build_fk_slope_mask(4e-4, 1/32, F, K)
FK_MASK             *= dash.build_fk_notch_mask(0, 128, K, NSAMPLES_TWIN)
gather.plot()

##### And you can trim the data

In [ ]:
gather.trim(starttime="2017-03-26T18:00:30.8Z", endtime="2017-03-26T18:00:31.5Z")
gather.plot()

In [ ]:
TWIN                 = pd.to_timedelta(4, unit="S")
BUFFER               = pd.to_timedelta(0.1, unit="S")
OUTPUT_NTRACES       = 3039
OUTPUT_SAMPLING_RATE = 1000

NSAMPLES_TWIN        = int((TWIN+2*BUFFER).total_seconds()*OUTPUT_SAMPLING_RATE)
F                    = np.fft.fftfreq(NSAMPLES_TWIN, d=1/OUTPUT_SAMPLING_RATE)
K                    = np.fft.fftfreq(OUTPUT_NTRACES)
FK_MASK              = dash.build_fk_slope_mask(4e-4, 1/32, F, K)
FK_MASK             *= dash.build_fk_notch_mask(0, 128, K, NSAMPLES_TWIN)

In [ ]:
plt.close("all")
ax = dash.plot_fk_mask(F, K, FK_MASK)
for i in range(0, K.shape[0], 100):
    ax.plot(
        F[i] + 2*np.fft.fftshift(FK_MASK[:, i]),
        np.fft.fftshift(K),
        linestyle=":",
        linewidth=0.5,
        color="w"
    )

In [ ]:
gather.plot()

#### DataFrames can be used for statistical processing
##### e.g., computing the STA/LTA

In [ ]:
plt.close("all")
fig, ax = plt.subplots()
ax.pcolorfast(
    gather.dataf.rolling(10).var() / gather.dataf.rolling(100).var()
)
ax.invert_yaxis()